In [4]:
import graph_tool.all as gt
import os

def build_dag_from_str(g_str):
    g_elements=g_str.split('\n')[:-1]
    g = gt.Graph(directed=True)
    status = g.new_vertex_property("int")
    ops = g.new_vertex_property("string")
    inputs = g.new_vertex_property("string")
    
    for g_e in g_elements:
        command, params = g_e.split(',', 1)

        if command == 'v':
            vid,operation, inputdir = params.split(',')
            v = g.add_vertex()
            status[v] = 0
            ops[v] = operation
            inputs[v] = inputdir
        elif command == 'e':
            src, dest = params.split(',')
            g.add_edge(src, dest)
    g.vp['status'] = status;
    g.vp['ops'] = ops;
    g.vp['inputdir'] = inputs
    return g;


def submit(g, ready=[]):
    print(ready)
    process = []
    for v in ready:
        program = g.vp['ops'][v]
        inputdir = g.vp['inputdir'][v]
        
        cur_dir = os.getcwd()
        cfg_hibench_root='/local0/HiBench'
        
        cfg_executable = 'bin/workloads/micro/%s/hadoop/run2.sh'%(program)
        
        process.append(subprocess.Popen([cfg.hibench_command, inputdir], 
                                        stdout=subprocess.PIPE, shell=True, executable="/bin/bash"))

    print('Wait for ', cfg.grace_time)
    time.sleep(cfg.grace_time)

    # get app name
    app = yarn.get_appname();

    exit_codes = [p.wait() for p in process]
    
    print(exit_codes)
    
    os.chdir(cfg_hibench_root)
        
    os.chdir(cur_dir)
    


def schedule(g):
    sort = gt.topological_sort(g)
    schedule = 1;
    to_be_executed = []
    for v in sort:
        # get in-degree neighbors
        schedule = 1; 
        in_neight_vrtx = g.get_in_neighbors(v)
        for nv in in_neight_vrtx:
            if g.vp.status[nv] == 0:
                schedule = 0;
        if schedule:
            to_be_executed.append(v) 
    return to_be_executed

with open('graphs.g', 'r') as fd:
    graph_strs = fd.read().split('#')[1:]
    
    for g_str in graph_strs:
        g = build_dag_from_str(g_str)
        
        to_be_executed = schedule(g)
        
        submit(g, to_be_executed)

[1, 0]
bin/workloads/micro/wordcount/hadoop/run2.sh s3a://data/HiBench/Wordcount/Input-1G-1
bin/workloads/micro/wordcount/hadoop/run2.sh s3a://data/HiBench/Wordcount/Input-1G
[3, 1, 0]
bin/workloads/micro/wordcount/hadoop/run2.sh 32G
bin/workloads/micro/wordcount/hadoop/run2.sh 32G
bin/workloads/micro/wordcount/hadoop/run2.sh 32G
